## AIM: Train a multiclass classification model for predicting the psychiatric diagnosis based on EEG features

We will start with training a Support Vector Classification model to predict psychiatric diagnosis with the variance in power per frequency band per channel group (frontal, central, parietal, occipital).

In [11]:
import numpy as np
import pandas as pd
import sklearn
import pickle
import mne
import os
import matplotlib.pyplot as plt
from mne.time_frequency import tfr_multitaper

%matplotlib inline

# prevent extensive logging
mne.set_log_level('WARNING')

### Loading in participants data
and excluding replication data

In [49]:
df_participants = pd.read_pickle('D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN_participants_V2_data\df_participants.pkl')
print(f'all participants: {df_participants.shape}')
df_participants = df_participants[df_participants['DISC/REP'] == 'DISCOVERY'] # select the discovery group
print(f'discovery participants: {df_participants.shape}')
df_participants[0:15]

all participants: (1347, 11)
discovery participants: (1227, 11)


,participants_ID,DISC/REP,indication,formal_status,Dataset,age,gender,sessID,nrSessions,EC,EO
120,sub-87963457,DISCOVERY,BURNOUT,BURNOUT,NaN,NaN,0,1,1,True,True
121,sub-87963593,DISCOVERY,BURNOUT,BURNOUT,NaN,52.90,1,1,1,True,True
122,sub-87963725,DISCOVERY,SMC,UNKNOWN,NaN,77.99,0,1,1,True,True
123,sub-87963769,DISCOVERY,SMC,UNKNOWN,NaN,83.30,1,1,1,True,True
124,sub-87964717,DISCOVERY,SMC,UNKNOWN,NaN,58.29,0,1,2,True,True
125,sub-87964717,DISCOVERY,SMC,UNKNOWN,NaN,61.06,0,3,2,True,True
126,sub-87965301,DISCOVERY,SMC,UNKNOWN,NaN,58.39,0,1,1,True,True
127,sub-87965393,DISCOVERY,BURNOUT,BURNOUT,NaN,50.78,1,1,1,True,True
128,sub-87965801,DISCOVERY,BURNOUT,BURNOUT,NaN,47.39,1,1,1,True,True
129,sub-87965889,DISCOVERY,BURNOUT,BURNOUT,NaN,51.15,1,1,1,True,True


### Creating montage and info object for PSD calculation with MNE

In [14]:
## Set montage based on channel names and locations provided in Van Dijk et al., (2022) (Copied from Anne van Duijvenbode)

ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg',\
           'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', \
           'eog', 'eog', 'eog', 'eog', 'ecg', 'eog', 'emg']

ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC3', 'FCz', 'FC4', 'T7', 'C3', 'Cz', 'C4', 'T8', 'CP3', \
            'CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2', 'VPVA', 'VNVB', 'HPHL', 'HNHR', 'Erbs', \
            'OrbOcc', 'Mass']

dict_eeg_channels =  {ch_names[i]: ch_types[i] for i in range(len(ch_types))}

dict_ch_pos = {'Fp1' : [-26.81, 84.06, -10.56],
               'Fp2' : [29.41, 83.74, -10.04],
               'F7'  : [-66.99, 41.69, -15.96],
               'F3'  : [-48.05, 51.87, 39.87],
               'Fz'  : [0.90, 57.01, 66.36],
               'F4'  : [50.38, 51.84, 41.33],
               'F8'  : [68.71, 41.16, -15.31],
               'FC3' : [-58.83, 21.02, 54.82],
               'FCz' : [0.57, 24.63, 87.63],
               'FC4' : [60.29, 21.16, 55.58], 
               'T7'  : [-83.36, -16.52, -12.65], 
               'C3'  : [-65.57, -13.25, 64.98],
               'Cz'  : [0.23, -11.28, 99.81],
               'C4'  : [66.50, -12.80, 65.11],
               'T8'  : [84.44, -16.65, -11.79], 
               'CP3' : [-65.51, -48.48, 68.57],
               'CPz' : [-0.42, -48.77, 98.37], 
               'CP4' : [65.03, -48.35, 68.57], 
               'P7': [-71.46, -75.17, -3.70], 
               'P3'  : [-55.07, -80.11, 59.44], 
               'Pz'  : [-0.87, -82.23, 82.43],
               'P4'  : [53.51, -80.13, 59.40], 
               'P8' : [71.10, -75.17, -3.69], 
               'O1'  : [-28.98, -114.52, 9.67],  
               'Oz'  : [-1.41, -117.79, 15.84],
               'O2'  : [26.89, -114.68, 9.45]
              }

dict_ch_pos_m = {'Fp1' : [-0.2681, 0.8406, -0.1056],
               'Fp2' : [0.2941, 0.8374, -0.1004],
               'F7'  : [-0.6699, 0.4169, -0.1596],
               'F3'  : [-0.4805, 0.5187, 0.3987],
               'Fz'  : [0.0090, 0.5701, 0.6636],
               'F4'  : [0.5038, 0.5184, 0.4133],
               'F8'  : [0.6871, 0.4116, -0.1531],
               'FC3' : [-0.5883, 0.2102, 0.5482],
               'FCz' : [0.0057, 0.2463, 0.8763],
               'FC4' : [0.6029, 0.2116, 0.5558], 
               'T7'  : [-0.8336, -0.1652, -0.1265], 
               'C3'  : [-0.6557, -0.1325, 0.6498],
               'Cz'  : [0.0023, -0.1128, 0.9981],
               'C4'  : [0.6650, -0.1280, 0.6511],
               'T8'  : [0.8444, -0.1665, -0.1179], 
               'CP3' : [-0.6551, -0.4848, 0.6857],
               'CPz' : [-0.042, -0.4877, 0.9837], 
               'CP4' : [0.6503, -0.4835, 0.6857], 
               'P7'  : [-0.7146, -0.7517, -0.0370], 
               'P3'  : [-0.5507, -0.8011, 0.5944], 
               'Pz'  : [-0.0087, -0.8223, 0.8243],
               'P4'  : [0.5351, -0.8013, 0.5940], 
               'P8'  : [0.7110, -0.7517, -0.0369], 
               'O1'  : [-0.2898, -1.1452, 0.0967],  
               'Oz'  : [-0.0141, -1.1779, 0.1584],
               'O2'  : [0.2689, -1.1468, 0.0945]
              }

dict_ch_pos_array = {'Fp1' : np.array([-0.02681, 0.08406, -0.01056]),
               'Fp2' : np.array([0.02941, 0.08374, -0.01004]),
               'F7'  : np.array([-0.06699, 0.04169, -0.01596]),
               'F3'  : np.array([-0.04805, 0.05187, 0.03987]),
               'Fz'  : np.array([0.00090, 0.05701, 0.06636]),
               'F4'  : np.array([0.05038, 0.05184, 0.04133]),
               'F8'  : np.array([0.06871, 0.04116, -0.01531]),
               'FC3' : np.array([-0.05883, 0.02102, 0.05482]),
               'FCz' : np.array([0.00057, 0.02463, 0.08763]),
               'FC4' : np.array([0.06029, 0.02116, 0.05558]), 
               'T7'  : np.array([-0.08336, -0.01652, -0.01265]), 
               'C3'  : np.array([-0.06557, -0.01325, 0.06498]),
               'Cz'  : np.array([0.000023, -0.01128, 0.09981]),
               'C4'  : np.array([0.06650, -0.01280, 0.06511]),
               'T8'  : np.array([0.08444, -0.01665, -0.01179]), 
               'CP3' : np.array([-0.06551, -0.04848, 0.06857]),
               'CPz' : np.array([-0.0042, -0.04877, 0.09837]), 
               'CP4' : np.array([0.06503, -0.04835, 0.06857]), 
               'P7'  : np.array([-0.07146, -0.07517, -0.00370]), 
               'P3'  : np.array([-0.05507, -0.08011, 0.05944]), 
               'Pz'  : np.array([-0.00087, -0.08223, 0.08243]),
               'P4'  : np.array([0.05351, -0.08013, 0.05940]), 
               'P8'  : np.array([0.07110, -0.07517, -0.00369]), 
               'O1'  : np.array([-0.02898, -0.11452, 0.00967]),  
               'Oz'  : np.array([-0.00141, -0.11779, 0.01584]),
               'O2'  : np.array([0.02689, -0.11468, 0.00945])
              }

# channel groupings
frontal = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC3', 'FCz', 'FC4']
central = ['T7', 'C3', 'Cz', 'C4', 'T8']
parietal = ['CP3','CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8'] 
occipital = ['O1', 'Oz', 'O2']
channel_groups = {'frontal': frontal, 'central': central, 'parietal': parietal, 'occipital': occipital}

# define (5) frequencies of interest for TFR per frequency band
delta = np.array([1, 1.5, 2, 2.5, 3]) # starting at one because of high-pass filter
theta = np.array([4, 4.75, 5.5, 6.25, 7])
alpha = np.array([8, 9, 10, 11, 12])
beta = np.array([13, 17.25, 21.5, 25.75, 30])
gamma = np.array([42, 54, 66, 78, 90]) 
bands = {'delta': delta, 'theta': theta, 'alpha': alpha, 'beta': beta, 'gamma': gamma}


## Create montage
montage = mne.channels.make_dig_montage(ch_pos = dict_ch_pos_array, coord_frame = 'head')

# Create info object for MNE
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=500)
info.set_montage(montage=montage, on_missing= 'raise')
print(info)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC3, FCz, FC4, T7, C3, Cz, C4, T8, ...
 chs: 26 EEG, 5 EOG, 1 ECG, 1 EMG
 custom_ref_applied: False
 dig: 29 items (3 Cardinal, 26 EEG)
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 500.0 Hz
>


### Feature extraction and storing in df

In [54]:
# calculate variance in power per freq band and per channel group for each file and store in dataframe
eeg_dir = "D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN-dataset-derivatives\derivatives\preprocessed"

#exlude_dirs = ['preprocessed', 'results_manuscript', 'adhd_sample'] # exclude these directories
sample_ids = df_participants['participants_ID'].tolist() # list of participants to include
#sample_ids = ['sub-87966293']

df_features = pd.DataFrame() # create empty dataframe to store features

# counter for progress
count = 1
if count == 1:
    total_files = 0
    for _, dirs, files in os.walk(eeg_dir):
        #dirs[:] = [d for d in dirs if d not in exlude_dirs] # exclude directories
        total_files += len([file for file in files if '.npy' in file])
    total_files -= len(sample_ids) * 2

for subdir, dirs, files in os.walk(eeg_dir): # iterate through all files
    #dirs[:] = [d for d in dirs if d not in exlude_dirs] # exclude directories
    for file in files:
        if any(sample_id in file for sample_id in sample_ids): # filter participants to include
            if 'ses-1' in file and '.npy' in file and 'BAD' not in file: # filter first session, .npy files, and non-bad files
                filepath = os.path.join(subdir, file) # path to eeg file

                # needs specific info object, because has one less channel
                info = mne.create_info(ch_names=ch_names[:32], ch_types=ch_types[:32], sfreq=500)
                info.set_montage(montage=montage, on_missing= 'raise')

                preprocessed_eeg = np.load(filepath, allow_pickle = True)
                raw = mne.io.RawArray(np.squeeze(preprocessed_eeg['data']), info)

                # epoch the data
                epochs = mne.make_fixed_length_epochs(raw, duration = 9.95, overlap = 0)

                if 'EC' in file:
                    cond = 'EC'
                if 'EO' in file:
                    cond = 'EO'

                # determine age, gender, and diagnosis of participant
                age = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'age'].values[0]
                gender = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'gender'].values[0]
                if df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'formal_status'].values[0] == 'UNKNOWN':
                    diagnosis = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'indication'].values[0]
                if df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'formal_status'].values[0] != 'UNKNOWN':
                    diagnosis = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'formal_status'].values[0]

                # add data to empty dictionary
                feature_dict = {}
                feature_dict['ID'] = [file.split('_')[0]] * epochs.get_data().shape[0]
                feature_dict['age'] = [age] * epochs.get_data().shape[0]
                feature_dict['gender'] = [gender] * epochs.get_data().shape[0]
                feature_dict['diagnosis'] = [diagnosis] * epochs.get_data().shape[0]
                feature_dict['EO/EC'] = [cond] * epochs.get_data().shape[0]
                feature_dict['epoch'] = list(range(1, epochs.get_data().shape[0] + 1))
                
                # calculate TFR per band and channel group using multitaper convolution
                for band in bands:
                    for group in channel_groups:
                        tfr_mt = tfr_multitaper(
                            epochs,
                            freqs=bands[band],
                            n_cycles=(bands[band] / 2),
                            time_bandwidth=4, # just random for now
                            use_fft=True,
                            return_itc=False,
                            average=False,
                            decim=8, # decim reduces sampling rate of the tf decomposition by the defined factor
                            n_jobs=-1,
                            picks=channel_groups[group])
                        tfr_mt.apply_baseline((None, None), mode='logratio') # baseline correction, (None, None) = whole epoch

                        # compute average spectral spower per epoch per channel group
                        tfr_mt_std = np.std(tfr_mt.data, axis = 3) # calculate stdev over time
                        tfr_mt_std_mean = np.mean(tfr_mt_std, axis = (1, 2)) # calculate mean of stdev over channels and frequencies

                        feature_dict[f'{group}_{band}'] = tfr_mt_std_mean # add to dictionary
                
                # add to dataframe
                df_features = pd.concat([df_features, pd.DataFrame(feature_dict)], ignore_index = True)

                print(f'\rProgress: {count}/{total_files} files processed.', end = '')
                count += 1
                
print(f'\n{df_features.shape}')
df_features.sample(5)

Progress: 2288/2692 files processed.
(27461, 26)


,ID,age,gender,diagnosis,EO/EC,epoch,frontal_delta,central_delta,parietal_delta,occipital_delta,...,parietal_alpha,occipital_alpha,frontal_beta,central_beta,parietal_beta,occipital_beta,frontal_gamma,central_gamma,parietal_gamma,occipital_gamma
23315,sub-88065425,44.62,0,MDD,EO,7,0.358983,0.346171,0.351030,0.383055,...,0.375147,0.392969,0.303941,0.287612,0.321925,0.344954,0.304924,0.279593,0.273020,0.271866
12111,sub-88025057,67.55,1,MDD,EO,10,0.388807,0.391471,0.363964,0.344105,...,0.354912,0.325065,0.300517,0.299131,0.289363,0.271069,0.270618,0.264079,0.264577,0.264958
7604,sub-88013629,10.97,1,NaN,EO,9,0.476117,0.394402,0.427574,0.379410,...,0.366636,0.316806,0.341467,0.342962,0.359685,0.383740,0.385861,0.383113,0.386890,0.454251
15171,sub-88038025,11.38,1,ADHD,EC,10,0.399910,0.415822,0.412045,0.366262,...,0.365943,0.349264,0.310514,0.301646,0.286953,0.304944,0.277846,0.269927,0.263425,0.270947
3797,sub-88005485,16.78,0,ADHD,EC,6,0.441210,0.399826,0.412588,0.446614,...,0.311833,0.297232,0.270807,0.284007,0.281161,0.305681,0.289532,0.286402,0.284343,0.285580


In [55]:
df_features.to_pickle('D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TD-BRAIN_extracted_features/df_std_features.pkl')

In [91]:
df_features.dtypes

ID                  object
age                float64
gender               int32
diagnosis           object
EO/EC               object
epoch                int64
frontal_delta      float64
central_delta      float64
parietal_delta     float64
occipital_delta    float64
frontal_theta      float64
central_theta      float64
parietal_theta     float64
occipital_theta    float64
frontal_alpha      float64
central_alpha      float64
parietal_alpha     float64
occipital_alpha    float64
frontal_beta       float64
central_beta       float64
parietal_beta      float64
occipital_beta     float64
frontal_gamma      float64
central_gamma      float64
parietal_gamma     float64
occipital_gamma    float64
dtype: object

In [104]:
df_features = pd.read_pickle('D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TD-BRAIN_extracted_features/df_std_features.pkl')
df_features['EO/EC'] = (df_features['EO/EC'] == 'EO').astype('int')
df_features['diagnosis'] = df_features['diagnosis'].astype('category')
df_features.sample(7)

,ID,age,gender,diagnosis,EO/EC,epoch,frontal_delta,central_delta,parietal_delta,occipital_delta,...,parietal_alpha,occipital_alpha,frontal_beta,central_beta,parietal_beta,occipital_beta,frontal_gamma,central_gamma,parietal_gamma,occipital_gamma
27377,sub-88078749,37.22,0,INSOMNIA,1,1,0.403476,0.328075,0.374296,0.393930,...,0.309027,0.299576,0.269556,0.271463,0.254627,0.260901,0.269438,0.271599,0.273250,0.270287
14850,sub-88035677,38.00,1,MDD,0,1,0.330494,0.352779,0.360977,0.388127,...,0.310154,0.326189,0.288945,0.271318,0.279525,0.285755,0.263849,0.269251,0.269067,0.274692
20284,sub-88053589,8.30,1,ADHD,0,12,0.461102,0.429050,0.416314,0.404076,...,0.326904,0.331926,0.315774,0.310331,0.287903,0.274776,0.444291,0.472984,0.396085,0.377890
1133,sub-87968361,56.13,0,SMC,1,6,0.391068,0.378987,0.387155,0.359594,...,0.428663,0.449647,0.286879,0.302547,0.309070,0.306727,0.263962,0.265733,0.288276,0.274592
8828,sub-88016601,9.55,1,NaN,1,9,0.358152,0.391335,0.380111,0.314724,...,0.267423,0.347187,0.270628,0.280052,0.274031,0.293612,0.277899,0.270875,0.270557,0.268579
9231,sub-88017453,19.84,1,NaN,1,4,0.361305,0.343602,0.354899,0.282072,...,0.364078,0.289084,0.273818,0.274769,0.264622,0.267996,0.270260,0.266839,0.271000,0.255712
14036,sub-88032753,13.00,1,ADHD,0,3,0.357172,0.384583,0.387069,0.375737,...,0.354654,0.332515,0.273663,0.288123,0.319110,0.324983,0.288015,0.289090,0.287206,0.284364


### Training the model (SVC)

In [111]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import GroupKFold, GridSearchCV, GroupShuffleSplit
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, cross_validate
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

df_features = df_features.dropna()

# assign groups (IDs) for GroupKFold
groups = df_features['ID'].tolist()
print(len(groups))

# define numeric & categorical columns
numeric_cols = []
for band in bands:
    for group in channel_groups:
        numeric_cols.append(f'{group}_{band}')
cat_cols = ['gender', 'EO/EC']

# # split data in train and test set, grouped by ID
# gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42) 
# for train_idx, test_idx in gss.split(df_features.drop(columns = 'diagnosis'), df_features['diagnosis'], groups = df_features['ID']):
#     train_set = df_features.iloc[train_idx]
#     test_set = df_features.iloc[test_idx]
# X_train, Y_train = train_set.drop(columns = ['ID', 'diagnosis', 'epoch']), train_set['diagnosis'] # is het probleem niet dat het model de specifieke leeftijd heel zwaar gaat wegen als predictor?
# X_test, Y_test = test_set.drop(columns = ['ID', 'diagnosis', 'epoch']), test_set['diagnosis']
X = df_features.drop(columns = ['ID', 'diagnosis', 'epoch'])
Y = df_features['diagnosis']

## Preprocessing steps for Pipeline for numeric features and categorical features
numeric_transformer = Pipeline(steps = [
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer(strategy='mean'))
])

categoric_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median'))
])

## Create ColumnTransformer to apply to any columnset
from sklearn.compose import ColumnTransformer
col_trans = ColumnTransformer(transformers=[
    ('numeric_transformer', numeric_transformer, numeric_cols),
    ('catergoric_transformer', categoric_transformer, cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

# Hyperparameter ranges for GridSearch
param_grid = {
'rbf_svm__C': [0.01, 0.1, 1, 10, 100],
'rbf_svm__gamma': [0.1, 1, 10, 100],
'rbf_svm__kernel': ['rbf'],
}

## Final pipeline
steps = [('col_trans', col_trans), ('rbf_svm', SVC(class_weight = 'balanced'))]
pipeline = Pipeline(steps)

# define both cross-validation objects (inner & outer)    
inner_cv = GroupKFold(n_splits = 5)
outer_cv = GroupKFold(n_splits = 3)

## Make f1-scorer
from sklearn.metrics import fbeta_score, make_scorer
f1_scorer = make_scorer(fbeta_score, beta = 1)

## Non-nested parameter search and scoring (f1-score)
gridsearch = GridSearchCV(pipeline, param_grid = param_grid, cv = outer_cv, scoring = f1_scorer, error_score = 'raise', return_train_score = True)
gridsearch_fit = gridsearch.fit(X, Y, groups = groups)
print(f'Best parameters: {gridsearch_fit.best_params_}')
print(f'Best f1-score: {gridsearch_fit.best_score_}')

## Nested CV with parameter optimization to compute testing score (f1-score)
svm_test_score = cross_val_score(gridsearch_fit, X = X, y = Y, cv = inner_cv, groups = groups, fit_params={"groups": groups}, scoring = f1_scorer, error_score = 'raise')
print('SVM test scores EO:', svm_test_score)
print('Mean SVM test score EO:', np.mean(svm_test_score))




20720


ValueError: pos_label=1 is not a valid label: It should be one of ['ADD' 'ADHD' 'ADHD ' 'ADHD/ANXIETY' 'ADHD/ASD/ANXIETY' 'ADHD/ASPERGER'
 'ADHD/DYSLEXIA' 'ADHD/DYSLEXIA/DYSCALCULIA' 'ADHD/EPILEPSY' 'ADHD/GTS'
 'ADHD/MDD' 'ADHD/PDD NOS' 'ANXIETY' 'ASD' 'ASD/ASPERGER' 'ASPERGER'
 'BIPOLAR' 'BURNOUT' 'CHRONIC PAIN' 'CONVERSION DX' 'DEPERSONALIZATION'
 'DYSPRAXIA' 'Dyslexia' 'HEALTHY' 'INSOMNIA' 'MDD' 'MDD/ADHD'
 'MDD/ADHD/ANOREXIA' 'MDD/ADHD/LYME' 'MDD/ANXIETY' 'MDD/ANXIETY/TINNITUS'
 'MDD/OCD' 'MDD/OCD/ADHD' 'MDD/TRAUMA' 'MIGRAINE' 'MSA-C' 'OCD' 'OCD/MDD'
 'PARKINSON' 'PDD NOS' 'PDD NOS/ADHD' 'PDD NOS/ANXIETY' 'PDD NOS/ASD'
 'PDD NOS/DYSLEXIA' 'SMC' 'STROKE ' 'TBI' 'TINNITUS' 'TINNITUS/MDD'
 'Tinnitus' 'WHIPLASH']